In [ ]:
# !pip install -q openai datasets
%load_ext dotenv
%dotenv

In [4]:
# import json
import numpy as np

from openai import AzureOpenAI
from datasets import load_dataset
from sklearn.metrics import classification_report
# from google.colab import userdata
from tqdm import tqdm
import os

In [5]:
azure_api_key = os.getenv('azure_api_key')
azure_api_endpoint = os.getenv('azure_endpoint')

In [6]:
client = AzureOpenAI(
  azure_endpoint = azure_api_endpoint,
  api_key=azure_api_key,
  api_version="2024-02-01"
)

In [7]:
model_name = 'gpt-35-turbo' # deployment name

**Examples and Gold Examples**

A set of examples and gold examples for sentiment classification of Amazon product reviews is hosted in a HuggingFace dataset. Let us load this data and take a look at the samples in this data.

In [10]:
amazon_reviews = load_dataset("vijayagrawal/spam-email-classification")

SSLError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs-us-1.hf.co', port=443): Max retries exceeded with url: /repos/81/27/81273c3291815455b60fb310e136b3a28de5a5d1ed44b2f2f2864ab4b7d8d6e1/02647c85f56264b8dffe31871c285c75b73030f0c352a64af9aa8441876ad68e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27examples-00000-of-00001.parquet%3B+filename%3D%22examples-00000-of-00001.parquet%22%3B&Expires=1728904474&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyODkwNDQ3NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzgxLzI3LzgxMjczYzMyOTE4MTU0NTViNjBmYjMxMGUxMzZiM2EyOGRlNWE1ZDFlZDQ0YjJmMmYyODY0YWI0YjdkOGQ2ZTEvMDI2NDdjODVmNTYyNjRiOGRmZmUzMTg3MWMyODVjNzViNzMwMzBmMGMzNTJhNjRhZjlhYTg0NDE4NzZhZDY4ZT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=YzmQxhZRfUPeCkoHMxu1gCmCPNeKECcKlWDbH9Fuazah4B4v1Jg3p9zchSFApuIB67cavBzm2Ck98ZHQ5v8yAh0mRMjoRVE3Va3h4HEw6oqY~u4~1G764tI2R67bR45W~TQN1MrVVOq9C~kK2ZQ1ScGQQdowYQmJohOVAN~inv2S

As is evident from the above output, the data set has 32 samples as examples and 32 samples as gold examples.

In [7]:
amazon_reviews_examples_df = amazon_reviews['examples'].to_pandas()
amazon_reviews_gold_examples_df = amazon_reviews['gold_examples'].to_pandas()

In [8]:
amazon_reviews_examples_df.shape, amazon_reviews_gold_examples_df.shape

((30, 2), (30, 2))

As the above outputs indicate, there are 32 examples and 32 gold examples. We will sample from the examples to create the few shot prompt and evaluate the prompt on all 32 gold examples.

In [15]:
amazon_reviews_examples_df.sample(4)
#amazon_reviews_examples_df

,content,label
0,CONGRATULATIONS! You have won a free iPhone 13...,spam
7,Can you pick up some milk on your way home? We...,ham
13,Happy birthday! Hope you have a wonderful day!,ham
15,The project deadline has been extended to next...,ham


In [16]:
#amazon_reviews_gold_examples_df

**Assembling the prompt**

In [19]:
system_message = """
Your task is to classify the provided email content input as either “spam” or “ham” (not spam).
Consider the following guidelines:
1.	If the email body contains phrases typically associated with spam (e.g., offers for quick money, urgent requests, suspicious links), label it as “spam.”
2.	If the email body appears to be from a trusted source and does not contain any suspicious content, label it as “ham” (not spam).
"""

In [20]:
few_shot_prompt = [{'role':'system', 'content': system_message}]

We need to iterate over the rows of the examples DataFrame to append these examples as `user` and `assistant` messages to the few-shot prompt. We achieve this using the `iterrows` method.

In [21]:
for index, row in amazon_reviews_examples_df.iterrows():
    print('Example content:')
    print(row[0])
    print('Example Label:')
    print(row[1])
    break

Example content:
CONGRATULATIONS! You have won a free iPhone 13! Click here to claim your prize now!
Example Label:
spam


<ipython-input-21-9984271c2c92>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[0])
<ipython-input-21-9984271c2c92>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[1])


Notice that the label is an integer. However, LLMs accept only strings. So we need to convert the integer label to a string label as we assemble the few-shot prompt. Let us assemble a few-shot prompt with 4 examples.

In [22]:
for index, row in amazon_reviews_examples_df.sample(4).iterrows():
    example_review = row[0]
    example_label = row[1]

    few_shot_prompt.append(
        {
            'role': 'user',
            'content': example_review
        }
    )

    few_shot_prompt.append(
        {
            'role': 'assistant',
            'content': str(example_label) # LLMs accept only string inputs
        }
    )

<ipython-input-22-adc2862a687c>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  example_review = row[0]
<ipython-input-22-adc2862a687c>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  example_label = row[1]


In [23]:
few_shot_prompt

[{'role': 'system',
  'content': '\nYour task is to classify the provided email content input as either “spam” or “ham” (not spam).\nConsider the following guidelines:\n1.\tIf the email body contains phrases typically associated with spam (e.g., offers for quick money, urgent requests, suspicious links), label it as “spam.”\n2.\tIf the email body appears to be from a trusted source and does not contain any suspicious content, label it as “ham” (not spam).\n'},
 {'role': 'user',
  'content': 'Team lunch at 12:30 PM today in the conference room.'},
 {'role': 'assistant', 'content': 'ham'},
 {'role': 'user', 'content': 'The kids are staying at Grandmas this weekend.'},
 {'role': 'assistant', 'content': 'ham'},
 {'role': 'user',
  'content': 'Make $5000 weekly working from home! No experience needed!'},
 {'role': 'assistant', 'content': 'spam'},
 {'role': 'user',
  'content': 'URGENT: Your inheritance of $5.5M is pending release!'},
 {'role': 'assistant', 'content': 'spam'}]

We now have 4 examples in the few shot prompt that is ready for use. Before we deploy this prompt, we need to get an estimate of the performance of this prompt. Here is where we use gold examples to estimate the accuracy.

## Evaluation

In [24]:
predictions, ground_truths = [], []

In [26]:
for index, row in tqdm(amazon_reviews_gold_examples_df.iterrows()):
    gold_review = row[0]
    gold_label = row[1]

    user_input = [{'role':'user', 'content': gold_review}]

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=few_shot_prompt + user_input,
            temperature=0
        )

        predictions.append(response.choices[0].message.content) # convert the string label back to int
        ground_truths.append(gold_label)
    except Exception as e:
        print(e) # Log error and continue
        continue

0it [00:00, ?it/s]<ipython-input-26-75d55713aa6b>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gold_review = row[0]
<ipython-input-26-75d55713aa6b>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  gold_label = row[1]
30it [00:06,  4.79it/s]


In [27]:
predictions = np.array(predictions)
ground_truths = np.array(ground_truths)
(predictions == ground_truths).mean()

1.0

The output above indicates that the accuracy of the few-shot prompt on gold examples. More fine-grained evaluation (e.g., F1 score) could also be used to establish the estimated accuracy of the prompt.

In [28]:
print(classification_report(ground_truths, predictions))

              precision    recall  f1-score   support

         ham       1.00      1.00      1.00        15
        spam       1.00      1.00      1.00        15

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



>More examples does not imply better accuracy. Increasing the number of examples in the few-shot prompt beyond 16 is not known to yield better performance.